In [230]:
from importlib import reload
import json
import pretty_midi
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

In [314]:
try:
    reload(midiMethods)
    from modules.midiMethods import *
except:
    print('reload failed')
    import modules.midiMethods as midiMethods
    from modules.midiMethods import *
try:
    reload(dataMethods)
    from modules.dataMethods import *
except:
    print('reload failed')
    import modules.dataMethods as dataMethods
    from modules.dataMethods import *


In [315]:
# This works just like this:
try:
    reload(modules.models)
except:
    print('reload failed')
    import modules.models as models

In [135]:
reload(dataMethods)
from modules.dataMethods import *

In [175]:
# test out my function for going to note_bin representation
pm = pretty_midi.PrettyMIDI('midi/basic_test.mid')
noteB2 = pm2note_bin(pm)
print(noteB2)
# pm2 = note_bin2pm(noteB2)
# pm2.write('midi/basic_test_tested.mid')

[[27, 0, 0, 3, 19, 27], [0, 0, 0, 3, 20, 28], [5, 3, 35, 3, 14, 25], [32, 0, 0, 3, 16, 26], [2, 3, 27, 0, 21, 17], [34, 0, 0, 1, 13, 19], [3, 0, 42, 0, 20, 11], [5, 0, 38, 0, 21, 11], [32, 0, 3, 1, 20, 13], [2, 0, 42, 0, 28, 9], [7, 0, 59, 2, 16, 8], [31, 0, 2, 2, 15, 5]]


In [75]:
maestro = pd.read_csv('training_data/maestro-v2.0.0withPeriod.csv', index_col=0)
filenames = list(maestro['midi_filename'])


In [316]:
data_path = 'C:/Users\Andrew/Documents/mlprojects/Datasets/MaestroV2.00/maestro-v2.0.0/'
examples = files2note_bin_examples(data_path, skip=100)
print(len(examples))

################# new example! Of length 4197
exceeded: 28
exceeded: 19
exceeded: 43
exceeded: 32
exceeded: 18
exceeded: 18
exceeded: 23
exceeded: 20
exceeded: 52
total_durations_exceeded:  9
total_shifts_exceeded:  0
################# new example! Of length 1223
total_durations_exceeded:  9
total_shifts_exceeded:  0
################# new example! Of length 4498
exceeded: 28
exceeded: 21
exceeded: 25
total_durations_exceeded:  12
total_shifts_exceeded:  0
################# new example! Of length 6072
total_durations_exceeded:  12
total_shifts_exceeded:  0
################# new example! Of length 2314
exceeded: 23
exceeded: 19
exceeded: 19
exceeded: 42
exceeded: 31
total_durations_exceeded:  17
total_shifts_exceeded:  0
################# new example! Of length 9734
exceeded: 51
exceeded: 41
exceeded: 62
exceeded: 59
exceeded: 26
exceeded: 53
exceeded: 51
exceeded: 32
exceeded: 63
exceeded: 49
exceeded: 30
exceeded: 13
total_durations_exceeded:  28
total_shifts_exceeded:  1
#############

In [317]:
# store examples in json format
with open('training_data/training_data_note_bin100.json', 'w') as f:
    json.dump(examples, f)

In [194]:
# test out what happens when we use no pedal on Berg sonata, convert to note_bin, and back to midi
pm = pretty_midi.PrettyMIDI('midi/BergOriginal.midi')
desus(pm)
note_bin2pm(pm2note_bin(pm)).write('midi/BergNoPedal.mid')

In [198]:
# could convert it to a tf dataset?
# use dataset.map() and dataset.filter() to apply transformations to each element
dataset = tf.data.Dataset.from_tensor_slices((X, Y))
dataset.element_spec

In [318]:
# build simple model
# excellent example of recurrent model here https://www.tensorflow.org/tutorials/text/text_generation
model1 = models.create_model1()


Model: "1layerLSTM"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_97 (InputLayer)           [(None, 128, 6)]     0                                            
__________________________________________________________________________________________________
lambda_106 (Lambda)             (None, 128, 88)      0           input_97[0][0]                   
__________________________________________________________________________________________________
lambda_107 (Lambda)             (None, 128, 10)      0           input_97[0][0]                   
__________________________________________________________________________________________________
lambda_108 (Lambda)             (None, 128, 60)      0           input_97[0][0]                   
_________________________________________________________________________________________

In [319]:
model1.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
X = np.array([example[:-1] for example in examples])
Y = np.array([example[1:] for example in examples])
# Y = np.reshape(Y, (len(examples), 6, 128))


model1.fit(X, [Y[:,:,i] for i in range(6)])

Train on 523 samples
523/523 [==============================] - 8s 14ms/sample - loss: 17.8955 - dense_60_loss: 4.4062 - dense_61_loss: 0.9573 - dense_62_loss: 3.7441 - dense_63_loss: 2.0394 - dense_64_loss: 3.4052 - dense_65_loss: 3.2697 - dense_60_accuracy: 0.0081 - dense_61_accuracy: 0.9155 - dense_62_accuracy: 0.0727 - dense_63_accuracy: 0.5134 - dense_64_accuracy: 0.0792 - dense_65_accuracy: 0.0672


In [310]:
[Y[:,:,i] for i in range(6)][4]

array([[28, 13, 13, ..., 13, 15, 14],
       [14, 15, 22, ..., 11,  4, 25],
       [17, 22, 18, ..., 18, 17, 10],
       ...,
       [11, 11, 29, ..., 13,  3,  2],
       [ 0, 29, 28, ..., 17, 11,  9],
       [ 9,  8,  3, ..., 29, 29, 29]])